In [8]:
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLTSVM
from sklearn.model_selection import GridSearchCV
import pickle

In [9]:
data_path='./data/'
training_filename = data_path + 'training_data.txt'
print('Loading training dataset...')
train_data = pd.read_table(training_filename, delimiter='*')
train_data.drop(columns=['tweets'],inplace=True)
# train_data = train_data.values.tolist()
print(len(train_data))
x_train_images=train_data['image_names'].values
y_train=train_data['hashtags'].values

Loading training dataset...
51172


/home/eric/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.


In [10]:
validation_filename = data_path + 'validation_data.txt'
validation_data = pd.read_table(validation_filename, delimiter='*')
validation_data.drop(columns=['tweets'],inplace=True)
# validation_data = validation_data.values.tolist()
# print(len(validation_data))
X_test_images=validation_data['image_names'].values
Y_test=validation_data['hashtags'].values

/home/eric/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [11]:
word_to_id_filename='word_to_id.p'
id_to_word_filename='id_to_word.p'
word_to_id = pickle.load(open(data_path +
                                           word_to_id_filename, 'rb'))
id_to_word = pickle.load(open(data_path +
                                           id_to_word_filename, 'rb'))

In [12]:
max_len=1003
def to_category_vector(texts,max_len):
    vector = np.zeros(max_len).astype(np.float32)
    for word in texts :
        vector[word_to_id[word]]=1.0
    return vector
document_Y = []
test_y=[]
for example in tqdm(y_train):
    arr=example.strip().split()
    document_Y.append(to_category_vector(arr,max_len))
document_Y=np.array(document_Y)
for example in tqdm(Y_test):
    arr=example.strip().split()
    test_y.append(to_category_vector(arr,max_len))
test_y=np.array(test_y)

100%|██████████| 5686/5686 [00:00<00:00, 158249.37it/s]


In [13]:
import h5py
import os
feature_path=os.path.join(data_path,'features.h5')
features = h5py.File(feature_path, 'r')

/home/eric/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [14]:
x_train=[]
for image_path in tqdm(x_train_images):
#     print(image_path)
    feat=features[image_path][:]
    x_train.append(feat)

100%|██████████| 51172/51172 [00:12<00:00, 4210.81it/s]


In [15]:
x_test=[]
for image_path in tqdm(X_test_images):
#     print(image_path)
    feat=features[image_path][:]
    x_test.append(feat)

100%|██████████| 5686/5686 [00:01<00:00, 4072.66it/s]


In [22]:
x_train=np.matrix(x_train)
x_test=np.array(x_test)

In [23]:
print(x_train[0])

[[  2.586004    -7.850414    19.55981      7.5457907  -14.418045
    0.31973976  -5.305423    -4.5446534  -22.902504    -8.39107
   14.722457    21.123611     8.640915   -17.3944     -17.661306
   15.083023   -15.937547     9.618949    -5.110028    11.973469
   -9.792797    -3.1549323  -13.932744   -12.396228    -5.204569
  -20.145369    -9.005281    -4.762278    -7.88161      0.07996935
    8.213088     6.5893874    9.67198     -0.8083064  -13.930093
   13.670356    -8.261702    20.167332    16.321033    -0.7470479
    0.8494476  -12.559278   -22.57898     -7.868537    -5.8988223
    5.611532    -3.9219985  -10.368643    19.191938   -18.87932
   15.55354     15.29472     -6.7232685  -21.333677     6.6736364
    7.5993004   -0.11270273  -1.7769763   -8.011534    -8.159487
   11.307922     9.014247    -9.682915    -4.4411836   17.237093
   18.510677     3.6192615    6.3653116  -21.386194     5.6924157
    4.5930605   10.730754   -12.383157     6.8690023    8.329338
    4.930588     0.72

In [26]:
# parameters = {'c_k': [2**i for i in range(-5, 5, 2)]}
# print(parameters)
# score = 'f1_macro'

# clf = GridSearchCV(MLTSVM(), parameters, scoring=score)
# clf.fit(x_train, document_Y)

# print (clf.best_params_, clf.best_score_)
# from scipy import sparse
# classifier = MLTSVM(c_k = 2**-1)
# classifier.fit(x_train, document_Y)
classifier = Pipeline([
    ('clf', OneVsRestClassifier(LinearSVC()))])

In [27]:
classifier.fit(x_train,document_Y)

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed t

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

...

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
 

/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1001 is present in all training examples.
  str(classes[c]))
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 1002 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
     steps=[('clf', OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None))])

In [28]:
predicted = classifier.predict(x_test)

In [30]:
print(predicted[0])

[0 0 0 ... 0 0 0]


In [32]:
with open("data/svm_predict.txt",'w') as f:
    for i in tqdm(range(predicted.shape[0])):
        g=np.where(predicted[i]==1)
        hashtags=[]
        for item in g[0]:
            word=id_to_word[item]
            hashtags.append(word)
        f.write(' '.join(hashtags))
        f.write('\n')

100%|██████████| 5686/5686 [00:00<00:00, 67982.91it/s]
